<a href="https://colab.research.google.com/github/angelfmo1001-cpu/Titanic-Modelo-Machine-Learning-/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de venta y demanda de Walmart.

## Librerias, paquetes para instalar y galeria de imagenes.

### Librerias y paquetes a instalar

In [ ]:
#Librerias importantes
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

#Coneccion con google drive
from google.colab import drive
from google.colab import files
#Importar la carpeta de drive
drive.mount('/content/drive') #coneccion con drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Galeria

In [ ]:
IMAGES_PATH = "Galeria"  # Nombre de la carpeta
os.makedirs(IMAGES_PATH, exist_ok=True) # Crea la carpeta si no existe

def save_fig(fig_id, tight_layout=True, fig_extension="pdf", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Imagen guardada como ", fig_id, "en la carpeta", IMAGES_PATH) #Mensaje mas informativo
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Carga y limpieza de la base de datos

### Carga de las bases de datos

In [ ]:
#Se cargan todos los archivos csv que hay
stores = pd.read_csv("/content/drive/MyDrive/Proyecto de ventas demanda/Base de datos/stores.csv")
features = pd.read_csv("/content/drive/MyDrive/Proyecto de ventas demanda/Base de datos/features/features.csv")
sales = pd.read_csv("/content/drive/MyDrive/Proyecto de ventas demanda/Base de datos/sampleSubmission/sampleSubmission.csv")
train = pd.read_csv("/content/drive/MyDrive/Proyecto de ventas demanda/Base de datos/train/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Proyecto de ventas demanda/Base de datos/test/test.csv")

### Vizualización de los datos

#### Stores

In [ ]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [ ]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [ ]:
len(stores) # Hay un total de 45 tiendas

45

In [ ]:
stores.columns

Index(['Store', 'Type', 'Size'], dtype='object')

In [ ]:
stores["Type"].value_counts()
#Las tiendas del tipo A aparecen con mayor frecuencia (22), seguidas de las oficinas tipo B (17) y en ultimo lugar las oficinas del tipo C (6)

,count
Type,
A,22
B,17
C,6


#### features

In [ ]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


Date se encuentra como objeto, se tiene que cambiar a data-time.

#### sales

In [ ]:
sales.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0


In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Id            115064 non-null  object
 1   Weekly_Sales  115064 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [ ]:
len(sales)

115064

## Análisis exploratorio de los datos